In [4]:
import pandas as pd
import re
from sqlalchemy import create_engine
from urllib.parse import quote
processing = create_engine("mysql://altrr_db:%s@192.168.0.32:1234/processing"% quote("abcd"))

In [6]:
df1=pd.read_sql('select * from tbl_mca_master_details_new',processing)
df1

,CIN,Company_Name,ROC_Code,Registration_Number,Company_Category,Company_SubCategory,Class_of_Company,Authorised_Capital_Rs,Paid_up_Capital_Rs,Number_of_Members,Date_of_Incorporation,Registered_Address,Address_other_than_R_o,Email_Id,Whether_Listed_or_not,Suspended_at_stock_exchange,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status_for_efiling,CrDt
0,U24239DL2003PTC118783,NUCLEUS PHARMACEUTICALS PRIVATE LIMITED,RoC-Delhi,118783,Company limited by Shares,Non-govt company,Private,100000,100000,0,04/02/2003,C-6/29SAFDARJUNG DEVELOPMENT AREA NEW DELHI DL...,-,None,Unlisted,-,-,-,Strike Off,None
1,U70101MH2003PTC139010,PRATHAMESH KRUPA BUILDCON PRIVATE LIMITED,RoC-Mumbai,139010,Company limited by Shares,Non-govt company,Private,900000,900000,0,04/02/2003,KW-59/7 AZAD NAGAR-2VEERA DESAI ROAD KRISHNA M...,-,shaileshkarekar@gmail.com,Unlisted,-,30/09/2023,31/03/2023,Active,None
2,U30007MH2003PTC139009,SHUBH MULTITRADE PRIVATE LIMITED,RoC-Mumbai,139009,Company limited by Shares,Non-govt company,Private,1500000,1225000,0,04/02/2003,"501, A, DHRUV BUILDING, GULMOHAR ROAD N.S.1, J...",-,sankalpsamal@gmail.com,Unlisted,-,30/09/2023,31/03/2023,Active,None
3,U27109MH2003PTC138995,SAINO ENGINEERING PRIVATE LIMITED,RoC-Mumbai,138995,Company limited by Shares,Non-govt company,Private,1000000,100000,0,04/02/2003,302 A-WING EVENING STAR BLDGRAHEJA VIHAR CHAND...,-,skbhageria@gmail.com,Unlisted,-,30/09/2013,31/03/2013,Strike Off,None
4,U28129MH2003PTC138997,VAISHALI JEWEL BOX PRIVATE LIMITED,RoC-Mumbai,138997,Company limited by Shares,Non-govt company,Private,1000000,140000,0,04/02/2003,37 LAXMI INDUSTRIAL ESTATEBANGUR NAGAR GOREGAO...,-,vaishalijewelbox@hotmail.com,Unlisted,-,30/09/2023,31/03/2023,Active,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344946,U18109WB2009PTC134768,CASUAL FASHIONS PRIVATE LIMITED,RoC-Kolkata,134768,Company limited by Shares,Non-govt company,Private,2500000,1051000,0,28/04/2009,"B/1/H/9, RAJA JANMENJOY ROAD BELIAGHATA Belegh...",-,spkochar@gmail.com,Unlisted,-,30/09/2024,31/03/2024,Active,None
344947,U22210OR2009PTC010703,CORNERSTONE MEDIA PRIVATE LIMITED,RoC-Cuttack,010703,Company limited by Shares,Non-govt company,Private,300000,100000,0,26/03/2009,"N-5/93,IRC VILLAGE NAYAPALLY BHUBANESWAR OR 75...",-,cornerstone.bbsr@gmail.com,Unlisted,-,30/09/2024,31/03/2024,Active,None
344948,U29268MH2009PTC193932,RENEWA ENVIROTECH PRIVATE LIMITED,RoC-Mumbai,193932,Company limited by Shares,Non-govt company,Private,100000,100000,0,09/07/2009,"PREETWELL, BARAGE ROAD, NEAR SUDHARSHAN VIHAR ...",-,v2kelkar@yahoo.com,Unlisted,-,30/09/2013,31/03/2013,Strike Off,None
344949,U31900DL2009PTC196254,SDB ELECTRICALS PRIVATE LIMITED,RoC-Delhi,196254,Company limited by Shares,Non-govt company,Private,9000000,9000000,0,20/11/2009,"CGF 10,DILKHUSH IND. ESTATE G.T.KARNAL ROAD,NE...",-,csgupta06@gmail.com,Unlisted,-,30/09/2024,31/03/2024,Active,None


In [10]:
df2=df1.copy()

In [11]:
df2['Registered_Address_temp']=df2['Registered_Address'].str.lower()

In [13]:
df2['Registered_Address_temp']=df2['Registered_Address'].apply(lambda x: x.lower() if isinstance(x,str) else x)

In [15]:
df2['state']=df2['Registered_Address_temp'].apply(lambda x: ', '.join(re.findall(r'\w+\s\d{6}',x)) if isinstance(x,str) else x)

In [16]:
df2['state']=df2['state'].apply(lambda x: x.split(', ')[-1] if isinstance(x,str) else x)

In [17]:
df2['pincode']=df2['Registered_Address_temp'].str.extract(r'(\d{6})')

In [18]:
df2['state']=df2['state'].str.strip()

In [19]:
df2['state']=df2['state'].apply(lambda x:re.sub(r'\d+','',x) if isinstance(x,str) else x)

In [20]:
df2['state']=df2['state'].apply(lambda x: None if isinstance(x,str) and len(x)==0 else x)

In [22]:
df2['state']=df2['state'].str.strip()

In [164]:
df2['state']=df2['state'].apply(lambda x:'uttarakhand' if x=='ur' else x)

In [165]:
words_to_replace={'an':'andaman & nicobar islands', 'ap':'andhra pradesh', 'ar':'arunachal pradesh', 'as':'assam', 'br':'bihar',
                 'ch':'chandigarh', 'ct':'chhattisgarh', 'dd':'daman and diu', 'dl':'delhi', 'dn':'dadra & nagar haveli',
                 'ga':'goa', 'gj':'gujarat', 'hp':'himachal pradesh', 'hr':'haryana', 'jh':'jharkhand', 'jk':'jammu and kashmir',
                 'ka':'karnataka', 'kl':'kerala', 'ld':'lakshadweep', 'lh':'(leh)jammu and kashmir', 'mh':'maharashtra',
                 'ml':'meghalaya', 'mp':'madhya pradesh', 'mn':'manipur', 'mz':'mizoram', 'nl':'nagaland', 'or':'odisha',
                 'pb':'punjab', 'py':'puducherry', 'rj':'rajasthan', 'tg':'telangana', 'tn':'tamil nadu', 'tr':'tripura',
                 'up':'uttar pradesh', 'ur':'uttarakhand'}

In [166]:
set(df2['state'])

{'(leh)jammu and kashmir',
 None,
 'andaman & nicobar islands',
 'andhra pradesh',
 'arunachal pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra & nagar haveli',
 'daman and diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal pradesh',
 'jammu and kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'lakshadweep',
 'madhya pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'tamil nadu',
 'telangana',
 'tripura',
 'uttar pradesh',
 'uttarakhand',
 'west bengal'}

In [163]:
check=df2[df2['Registered_Address_temp'].str.contains(' ur ',na=False)]
set(check['Registered_Address_temp'])

{'ujjwal maharani baghdehradun dehradun na uttaranchal ur 248006 in',
 'paradise tower, 3rd floor, 23 industrial area, patel nagar dehradun dehradun ur 248001 in',
 'c/o mohan chandra bhatt, bhola datt gic road, didihat na pithoragarh pithoragarh ur 262551 in',
 '22 civil lines mittal pan bhandar wali gali, roorkee haridwar haridwar ur 247667 in',
 'm/s universal industries b-8 industrial estate, bazpur road na kashipur udham singh nagar ur 244713 in',
 '98 and 98/1, ground floor canal road, jakhan rajpur (dehradun) dehradun dehradun ur 248009 in',
 'shop no. 1 chakrata road, near bindal bridge dehradun ur 248001 in',
 'h.no.-3/1262 bhotia paraw purnagiri park ,subhash nagar, haldwani nainital ur 263139 in',
 'c/o geeta singh naguthagu khatima ur 262308 in',
 'plot no 29b, shiv ganga industrial khasra no 82, estate village lakeshwari na bhagwanpur haridwar ur 247661 in',
 '53, sukrad, someshwar na almora almora ur 263637 in',
 'a-04, shri chandracharya appartment, nr. bhairav mandir, k

In [167]:
df2['state'].value_counts()

state
maharashtra                  66962
delhi                        51531
west bengal                  28831
tamil nadu                   25748
uttar pradesh                24830
karnataka                    23106
telangana                    22144
gujarat                      18663
rajasthan                    11628
kerala                       10976
haryana                      10913
madhya pradesh                7899
bihar                         7739
andhra pradesh                5760
odisha                        5457
punjab                        5055
jharkhand                     2940
assam                         2627
chhattisgarh                  2235
chandigarh                    2171
uttarakhand                   2017
jammu and kashmir             1471
goa                           1233
himachal pradesh              1126
puducherry                     461
manipur                        298
meghalaya                      276
tripura                        188
arunachal prad

In [11]:
# new=pd.read_excel(r'C:\Users\Administrator\Downloads\District_Masters.xlsx',sheet_name='Sheet2')

In [168]:
districts=['nicobars',  'north and middle andaman',  'south andaman',  'anantapur',  'chittoor',  'east godavari',  'guntur',  'kurnool',  'prakasam',
 'sri potti sriramulu nellore',  'srikakulam',  'visakhapatnam',  'vizianagaram',  'west godavari',  'y.s.r.',  'anjaw',  'changlang',
 'dibang valley',  'east kameng',  'east siang',  'kra daadi',  'kurung kumey',  'lohit',  'lower dibang valley',  'lower siang',
 'lower subansiri',  'namsai',  'papum pare',  'siang',  'tawang',  'tirap',  'upper siang',  'upper subansiri',  'west kameng',
 'west siang',  'baksa',  'barpeta',  'biswanath',  'bongaigaon',  'cachar',  'charaideo',  'chirang',  'darrang',  'dhemaji',
 'dhubri',  'dibrugarh',  'dima hasao',  'goalpara',  'golaghat',  'hailakandi',  'hojai',  'jorhat',  'kamrup',  'kamrup metropolitan',
 'karbi anglong',  'karimganj',  'kokrajhar',  'lakhimpur',  'majuli',  'morigaon',  'nagaon',  'nalbari',  'sivasagar',  'sonitpur',
 'south salamara-mankachar',  'tinsukia',  'udalguri',  'west karbi anglong',  'araria',  'arwal',  'aurangabad',  'banka',  'begusarai',
 'bhagalpur',  'bhojpur',  'buxar',  'darbhanga',  'gaya',  'gopalganj',  'jamui',  'jehanabad',  'kaimur (bhabua)',  'katihar',  'khagaria',
 'kishanganj',  'lakhisarai',  'madhepura',  'madhubani',  'munger',  'muzaffarpur',  'nalanda',  'nawada',  'pashchim champaran',  'patna',
 'purbi champaran',  'purnia',  'rohtas',  'saharsa',  'samastipur',  'saran',  'sheikhpura',  'sheohar',  'sitamarhi',  'siwan',  'supaul',
 'vaishali',  'chandigarh',  'balod',  'baloda bazar',  'balrampur',  'bastar',  'bemetara',  'bijapur',  'bilaspur',  'dakshin bastar dantewada',
 'dhamtari',  'durg',  'gariyaband',  'janjgir - champa',  'jashpur',  'kabeerdham',  'kondagaon',  'korba',  'koriya',  'mahasamund',  'mungeli',
 'narayanpur',  'raigarh',  'raipur',  'rajnandgaon',  'sukma',  'surajpur',  'surguja',  'uttar bastar kanker',  'dadra and nagar haveli',  'daman',
 'diu',  'north goa',  'south goa',  'ahmadabad',  'amreli',  'anand',  'arvalli',  'banas kantha',  'bharuch',  'bhavnagar',  'botad',
 'chhota udepur',  'devbhoomi dwarka',  'dohad',  'gandhinagar',  'gir somnath',  'jamnagar',  'junagadh',  'kachchh',  'kheda',  'mahesana',
 'mahisagar',  'morbi',  'narmada',  'navsari',  'panch mahals',  'patan',  'porbandar',  'rajkot',  'sabar kantha',  'surat',  'surendranagar',
 'tapi',  'the dangs',  'vadodara',  'valsad',  'ambala',  'bhiwani',  'charkhi dadri',  'faridabad',  'fatehabad',  'gurgaon',  'hisar',
 'jhajjar',  'jind',  'kaithal',  'karnal',  'kurukshetra',  'mahendragarh',  'mewat',  'palwal',  'panchkula',  'panipat',  'rewari',
 'rohtak',  'sirsa',  'sonipat',  'yamunanagar',  'chamba',  'hamirpur',  'kangra',  'kinnaur',  'kullu',  'lahul spiti',  'mandi',
 'shimla',  'sirmaur',  'solan',  'una',  'anantnag',  'badgam',  'bandipore',  'baramula',  'doda',  'ganderbal',  'jammu',
 'kargil',  'kathua',  'kishtwar',  'kulgam',  'kupwara',  'leh(ladakh)',  'pulwama',  'punch',  'rajouri',  'ramban',  'reasi',
 'samba',  'shupiyan',  'srinagar',  'udhampur',  'bokaro',  'chatra',  'deoghar',  'dhanbad',  'dumka',  'garhwa',  'giridih',
 'godda',  'gumla',  'hazaribagh',  'jamtara',  'khunti',  'kodarma',  'latehar',  'lohardaga',  'pakur',  'palamu',  'pashchimi singhbhum',
 'purbi singhbhum',  'ramgarh',  'ranchi',  'sahibganj',  'saraikela-kharsawan',  'simdega',  'bagalkot',  'bangalore',  'bangalore rural',
 'belgaum',  'bellary',  'bidar',  'chamarajanagar',  'chikkaballapura',  'chikmagalur',  'chitradurga',  'dakshina kannada',  'davanagere',
 'dharwad',  'gadag',  'gulbarga',  'hassan',  'haveri',  'kodagu',  'kolar',  'koppal',  'mandya',  'mysore',  'raichur',  'ramanagara',
 'shimoga',  'tumkur',  'udupi',  'uttara kannada',  'yadgir',  'alappuzha',  'ernakulam',  'idukki',  'kannur',  'kasaragod',  'kollam',
 'kottayam',  'kozhikode',  'malappuram',  'palakkad',  'pathanamthitta',  'thiruvananthapuram',  'thrissur',  'wayanad',  'lakshadweep',
 'agar malwa',  'alirajpur',  'anuppur',  'ashoknagar',  'balaghat',  'barwani',  'betul',  'bhind',  'bhopal',  'burhanpur',  'chhatarpur',
 'chhindwara',  'damoh',  'datia',  'dewas',  'dhar',  'dindori',  'guna',  'gwalior',  'harda',  'hoshangabad',  'indore',  'jabalpur',
 'jhabua',  'katni',  'khandwa (east nimar)',  'khargone (west nimar)',  'mandla',  'mandsaur',  'morena',  'narsimhapur',  'neemuch',
 'panna',  'raisen',  'rajgarh',  'ratlam',  'rewa',  'sagar',  'satna',  'sehore',  'seoni',  'shahdol',  'shajapur',  'sheopur',
 'shivpuri',  'sidhi',  'singrauli',  'tikamgarh',  'ujjain',  'umaria',  'vidisha',  'ahmadnagar',  'akola',  'amravati',  'bhandara',
 'bid',  'buldana',  'chandrapur',  'dhule',  'gadchiroli',  'gondiya',  'hingoli',  'jalgaon',  'jalna',  'kolhapur',  'latur',
 'mumbai',  'mumbai suburban',  'nagpur',  'nanded',  'nandurbar',  'nashik',  'osmanabad',  'palghar',  'parbhani',  'pune',
 'ratnagiri',  'sangli',  'satara',  'sindhudurg',  'solapur',  'thane',  'wardha',  'washim',  'yavatmal',  'bishnupur',
 'chandel',  'churachandpur',  'imphal east',  'imphal west',  'jiribam',  'kakching',  'kamjong',  'kangpokpi',  'noney',
 'pherzawl',  'senapati',  'tamenglong',  'tengnoupal',  'thoubal',  'ukhrul',  'east garo hills',  'east jaintia hills',
 'east khasi hills',  'jaintia hills',  'north garo hills',  'ribhoi',  'south garo hills',  'south west garo hills',
 'south west khasi hills',  'west garo hills',  'west jaintia hills',  'west khasi hills',  'aizawl',  'champhai',
 'kolasib',  'lawngtlai',  'lunglei',  'mamit',  'saiha',  'serchhip',  'dimapur',  'kiphire',  'kohima',  'longleng',
 'mokokchung',  'mon',  'peren',  'phek',  'tuensang',  'wokha',  'zunheboto',  'central',  'new delhi',  'shahdara',
 'south east delhi',  'anugul', 'balangir',  'baleshwar',  'bargarh',  'baudh',  'bhadrak',  'cuttack',  'debagarh', 'dhenkanal', 'gajapati',
 'ganjam',  'jagatsinghapur',  'jajapur',  'jharsuguda',  'kalahandi',  'kandhamal',  'kendrapara',  'kendujhar',
 'khordha',  'koraput',  'malkangiri',  'mayurbhanj',  'nabarangapur',  'nayagarh',  'nuapada',  ' puri ',  'rayagada',
 'sambalpur',  'subarnapur',  'sundargarh',  'karaikal',  'mahe',  'puducherry',  'yanam',  'amritsar',  'barnala',
 'bathinda',  'faridkot',  'fatehgarh sahib',  'fazilka',  'firozpur',  'gurdaspur',  'hoshiarpur',  'jalandhar',
 'kapurthala',  'ludhiana',  'mansa',  'moga',  'muktsar',  'pathankot',  'patiala',  'rupnagar',  'sahibzada ajit singh nagar',
 'sangrur',  'shahid bhagat singh nagar',  'tarn taran',  'ajmer',  'alwar',  'banswara',  'baran',  'barmer',  'bharatpur',
 'bhilwara',  'bikaner',  'bundi',  'chittaurgarh',  'churu',  'dausa',  'dhaulpur',  'dungarpur',  'hanumangarh',  'jaipur',
 'jaisalmer',  'jalor',  'jhalawar',  'jhunjhunun',  'jodhpur',  'karauli',  'kota',  'nagaur',  'pali',  'pratapgarh',
 'rajsamand',  'sawai madhopur',  'sikar',  'sirohi',  'sri ganganagar',  'tonk',  'udaipur',  'east district',
 'north  district',  'south district',  'west district',  'ariyalur',  'chennai',  'coimbatore',  'cuddalore',  'dharmapuri',
 'dindigul',  'erode',  'kancheepuram',  'kanniyakumari',  'karur',  'krishnagiri',  'madurai',  'nagapattinam',  'namakkal',
 'perambalur',  'pudukkottai',  'ramanathapuram',  'salem',  'sivaganga',  'thanjavur',  'the nilgiris',  'theni',  'thiruvallur',
 'thiruvarur',  'thoothukkudi',  'tiruchirappalli',  'tirunelveli',  'tiruppur',  'tiruvannamalai',  'vellore',  'viluppuram',
 'virudhunagar',  'adilabad',  'bhadradri',  'hyderabad',  'jagtial',  'jangaon',  'jayashankar',  'jogulamba',  'kamareddy',
 'karimnagar',  'khammam',  'komaram bheem',  'mahabubabad',  'mahbubnagar',  'mancherial',  'medak',  'medchal-malkajgiri',
 'nagarkurnool',  'nalgonda',  'nirmal',  'nizamabad',  'peddapalli',  'rajanna',  'rangareddy',  'sangareddy',  'siddipet',
 'suryapet',  'vikarabad',  'wanaparthy',  'warangal rural',  'warangal urban',  'yadadri',  'dhalai',  'gomati',  'khowai',
 'north tripura',  'sepahijala',  'south tripura',  'unakoti',  'west tripura',  'agra',  'aligarh',  'allahabad',  'ambedkar nagar',
 'amethi',  'amroha',  'auraiya',  'azamgarh',  'baghpat',  'bahraich',  'ballia',  'banda',  'bara banki',  'bareilly',  'basti',
 'bhadohi',  'bijnor',  'budaun',  'bulandshahr',  'chandauli',  'chitrakoot',  'deoria',  'etah',  'etawah',  'faizabad',  'farrukhabad',
 'fatehpur',  'firozabad',  'gautam buddha nagar',  'ghaziabad',  'ghazipur',  'gonda',  'gorakhpur',  'hapur',  'hardoi',  'hathras',
 'jalaun',  'jaunpur',  'jhansi',  'kannauj',  'kanpur dehat',  'kanpur nagar',  'kasganj',  'kaushambi',  'kheri',  'kushinagar',
 'lalitpur',  'lucknow',  'mahoba',  'mahrajganj',  'mainpuri',  'mathura',  ' mau ',  'meerut',  'mirzapur',  'moradabad',  'muzaffarnagar',
 'pilibhit',  'rae bareli','raebareli',  'rampur',  'saharanpur',  'sambhal',  'sant kabir nagar',  'shahjahanpur',  'shamli',  'shrawasti',
 'siddharthnagar',  'sitapur',  'sonbhadra',  'sultanpur',  'unnao',  'varanasi',  'almora',  'bageshwar',  'chamoli',  'champawat',
 'dehradun',  'garhwal',  'hardwar',  'nainital',  'pithoragarh',  'rudraprayag',  'tehri garhwal',  'udham singh nagar',  'uttarkashi',
 'alipurduar',  'bankura',  'barddhaman',  'birbhum',  'dakshin dinajpur',  'darjiling',  'haora',  'hugli',  'jalpaiguri',  'jhargram',
 'kalimpong',  'koch bihar',  'kolkata',  'maldah',  'murshidabad',  'nadia',  'north 24 parganas',  'parganas north',  '24 parganas',
 'paschim bardhaman',  'paschim medinipur',  'purba bardhaman',  'purba medinipur',  'puruliya',  'south 24 parganas',  'uttar dinajpur',  'krishna']

In [169]:
found=[]
for i in df2['Registered_Address_temp']:
        if isinstance(i,str):
            a=''
            for j in districts:
                if j in i:
                    a=j
                    found.append(j)
                    break
                else:
                    pass
            if len(a)!=0:
                pass
            else:
                found.append(None)
        else:
            found.append(None)

In [170]:
df2['district']=found

In [171]:
df2['district'].value_counts()

district
mumbai           30539
new delhi        27800
bangalore        18999
kolkata          17941
hyderabad        16800
                 ...  
tuensang             1
chhota udepur        1
y.s.r.               1
peren                1
baudh                1
Name: count, Length: 639, dtype: int64

In [727]:
df1['district'].value_counts()

district
mumbai              119723
new delhi           118514
bangalore            95376
hyderabad            72114
kolkata              71319
                     ...  
kondagaon                1
kabeerdham               1
longleng                 1
north garo hills         1
yadadri                  1
Name: count, Length: 659, dtype: int64

In [715]:
df1['Registered_Address_temp'][2]

'teachers colony g t road, bewar, mainpuri, mainpuri, uttar pradesh, india, 205301'

In [172]:
df2['state_backup']=df2['state']

In [173]:
a=df2[df2['district'].isnull()]
set(a['Registered_Address_temp'])

{'7th floor, minerva complex, 94, s.d road, secunderabad tg 500003 in',
 'd-335, g/f west vinod nagar, near tulsi dairy delhi east delhi dl 110092 in',
 '15/96civil lines kanpur uttar pradesh up 000000 in',
 '3, adisanpet street kanchipuram tn 631501 in',
 'lg-1 & 3 r-23 nehru enclave dl 110019 in',
 'no. 3 government colonyenglish bazar englishbazar wb 732101 in',
 'poovarnathalaivilai veedu,athencode, padanthalumoodu.p.o. k k district kanyakumari tn 629194 in',
 'h-6, bombay road chamrail, unit-v howrah howrah wb 711114 in',
 '89a ground floor dda janta flats village pitampura delhi dl 110034 in',
 'a 55/12, gf, chhakarpur dlf phase 1, sector 28 gurugram hr 122001 in',
 '880/804-b, main market rani bagh delhi dl 110034 in',
 'off at plot no-2-b-1, block unit no-302, t/f floor, lsc janakpuri, tff na delhi west delhi dl 110058 in',
 'h n.583 gram amgaon bada tehsil kareli dist narsingpur narsingpur narsinghpur mp 487225 in',
 'f.no. 3593, sector d, pkt 3, vasant kunj delhi dl 110070 in

In [175]:
df2['state'].value_counts()

state
maharashtra                  66962
delhi                        51531
west bengal                  28831
tamil nadu                   25748
uttar pradesh                24830
karnataka                    23106
telangana                    22144
gujarat                      18663
rajasthan                    11628
kerala                       10976
haryana                      10913
madhya pradesh                7899
bihar                         7739
andhra pradesh                5760
odisha                        5457
punjab                        5055
jharkhand                     2940
assam                         2627
chhattisgarh                  2235
chandigarh                    2171
uttarakhand                   2017
jammu and kashmir             1471
goa                           1233
himachal pradesh              1126
puducherry                     461
manipur                        298
meghalaya                      276
tripura                        188
arunachal prad

In [313]:
# check full state names one by one in address for null states as there might not be short forms available in the address
found=[]
for i,row in df2.iterrows():
    # state=row['state'] run for the first state only then run state_new
    state=row['state_new']
    address=row['Registered_Address_temp']
    if state==None:
        if isinstance(address,str):
            if 'lakshadweep' in address:
                found.append('lakshadweep')
            else:
                found.append(None)
        else:
            found.append(None)
    else:
        found.append(state)

In [314]:
len(found)

344951

In [315]:
df2['state_new']=found

In [319]:
df2['state_new'].value_counts()

state_new
Maharashtra                  66963
Delhi                        51531
West Bengal                  28836
Tamil Nadu                   25756
Uttar Pradesh                24830
Karnataka                    23106
Telangana                    22144
Gujarat                      18664
Rajasthan                    11628
Kerala                       10976
Haryana                      10913
Madhya Pradesh                7899
Bihar                         7739
Andhra Pradesh                5760
Odisha                        5457
Punjab                        5055
Jharkhand                     2940
Assam                         2627
Chhattisgarh                  2235
Chandigarh                    2171
Uttarakhand                   2017
Jammu And Kashmir             1472
Goa                           1233
Himachal Pradesh              1126
Puducherry                     461
Manipur                        298
Meghalaya                      276
Tripura                        188
Arunachal 

In [317]:
df2['state_new']=df2['state_new'].str.title()

In [318]:
df3=df2.copy()

In [329]:
df2

,CIN,Company_Name,ROC_Code,Registration_Number,Company_Category,Company_SubCategory,Class_of_Company,Authorised_Capital_Rs,Paid_up_Capital_Rs,Number_of_Members,...,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status_for_efiling,CrDt,Registered_Address_temp,state,pincode,district,state_backup,state_new
0,U24239DL2003PTC118783,NUCLEUS PHARMACEUTICALS PRIVATE LIMITED,RoC-Delhi,118783,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,-,-,Strike Off,None,c-6/29safdarjung development area new delhi dl...,delhi,110016,new delhi,delhi,Delhi
1,U70101MH2003PTC139010,PRATHAMESH KRUPA BUILDCON PRIVATE LIMITED,RoC-Mumbai,139010,Company limited by Shares,Non-govt company,Private,900000,900000,0,...,30/09/2023,31/03/2023,Active,None,kw-59/7 azad nagar-2veera desai road krishna m...,maharashtra,400053,mandi,maharashtra,Maharashtra
2,U30007MH2003PTC139009,SHUBH MULTITRADE PRIVATE LIMITED,RoC-Mumbai,139009,Company limited by Shares,Non-govt company,Private,1500000,1225000,0,...,30/09/2023,31/03/2023,Active,None,"501, a, dhruv building, gulmohar road n.s.1, j...",maharashtra,400049,mumbai,maharashtra,Maharashtra
3,U27109MH2003PTC138995,SAINO ENGINEERING PRIVATE LIMITED,RoC-Mumbai,138995,Company limited by Shares,Non-govt company,Private,1000000,100000,0,...,30/09/2013,31/03/2013,Strike Off,None,302 a-wing evening star bldgraheja vihar chand...,maharashtra,400072,mumbai,maharashtra,Maharashtra
4,U28129MH2003PTC138997,VAISHALI JEWEL BOX PRIVATE LIMITED,RoC-Mumbai,138997,Company limited by Shares,Non-govt company,Private,1000000,140000,0,...,30/09/2023,31/03/2023,Active,None,37 laxmi industrial estatebangur nagar goregao...,maharashtra,400090,mumbai,maharashtra,Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344946,U18109WB2009PTC134768,CASUAL FASHIONS PRIVATE LIMITED,RoC-Kolkata,134768,Company limited by Shares,Non-govt company,Private,2500000,1051000,0,...,30/09/2024,31/03/2024,Active,None,"b/1/h/9, raja janmenjoy road beliaghata belegh...",west bengal,700010,kolkata,west bengal,West Bengal
344947,U22210OR2009PTC010703,CORNERSTONE MEDIA PRIVATE LIMITED,RoC-Cuttack,010703,Company limited by Shares,Non-govt company,Private,300000,100000,0,...,30/09/2024,31/03/2024,Active,None,"n-5/93,irc village nayapally bhubaneswar or 75...",odisha,751015,None,odisha,Odisha
344948,U29268MH2009PTC193932,RENEWA ENVIROTECH PRIVATE LIMITED,RoC-Mumbai,193932,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,30/09/2013,31/03/2013,Strike Off,None,"preetwell, barage road, near sudharshan vihar ...",maharashtra,421503,dhar,maharashtra,Maharashtra
344949,U31900DL2009PTC196254,SDB ELECTRICALS PRIVATE LIMITED,RoC-Delhi,196254,Company limited by Shares,Non-govt company,Private,9000000,9000000,0,...,30/09/2024,31/03/2024,Active,None,"cgf 10,dilkhush ind. estate g.t.karnal road,ne...",delhi,110033,karnal,delhi,Delhi


In [733]:
engine = create_engine("mysql://altrr_db:%s@192.168.0.20:3306/fdb"% quote("altrr@2025"))

In [722]:
df1['Registered_Address_temp'][0]

'c/o neelam pandey w/o shri krishna pandey saurmau, sultanpur, sultanpur, uttar pradesh, india, 228001'

In [366]:
df_final=df2[['CIN','state_new','district','pincode']]

In [367]:
df_final

,CIN,state_new,district,pincode
0,U24239DL2003PTC118783,Delhi,new delhi,110016
1,U70101MH2003PTC139010,Maharashtra,mandi,400053
2,U30007MH2003PTC139009,Maharashtra,mumbai,400049
3,U27109MH2003PTC138995,Maharashtra,mumbai,400072
4,U28129MH2003PTC138997,Maharashtra,mumbai,400090
...,...,...,...,...
344946,U18109WB2009PTC134768,West Bengal,kolkata,700010
344947,U22210OR2009PTC010703,Odisha,None,751015
344948,U29268MH2009PTC193932,Maharashtra,dhar,421503
344949,U31900DL2009PTC196254,Delhi,karnal,110033


In [368]:
df_final=df_final.rename(columns={'state_new':'state'})

In [369]:
df_final['district']=df_final['district'].str.title()
df_final

,CIN,state,district,pincode
0,U24239DL2003PTC118783,Delhi,New Delhi,110016
1,U70101MH2003PTC139010,Maharashtra,Mandi,400053
2,U30007MH2003PTC139009,Maharashtra,Mumbai,400049
3,U27109MH2003PTC138995,Maharashtra,Mumbai,400072
4,U28129MH2003PTC138997,Maharashtra,Mumbai,400090
...,...,...,...,...
344946,U18109WB2009PTC134768,West Bengal,Kolkata,700010
344947,U22210OR2009PTC010703,Odisha,None,751015
344948,U29268MH2009PTC193932,Maharashtra,Dhar,421503
344949,U31900DL2009PTC196254,Delhi,Karnal,110033


In [370]:
df_final=df_final.drop_duplicates(subset='CIN')
len(df_final)

344475

In [372]:
empty_pin=df2[df2['pincode'].str.contains('000000',na=True)]

In [375]:
empty_pin

,CIN,Company_Name,ROC_Code,Registration_Number,Company_Category,Company_SubCategory,Class_of_Company,Authorised_Capital_Rs,Paid_up_Capital_Rs,Number_of_Members,...,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status_for_efiling,CrDt,Registered_Address_temp,state,pincode,district,state_backup,state_new
18,U51505MP2003PLC015556,ZENET CONCEPTS INDIA LIMITED,RoC-Gwalior,015556,Company limited by Shares,Non-govt company,Public,500000,244000,0,...,-,-,Strike Off,None,"m-1,mayur plaza,mayur market thatipur gwalior ...",madhya pradesh,000000,gwalior,madhya pradesh,Madhya Pradesh
20,U18101RJ2003PTC018079,SILVER FAB SUITING PRIVATE LIMITED,RoC-Jaipur,018079,Company limited by Shares,Non-govt company,Private,20000000,15305000,0,...,30/09/2024,31/03/2024,Active,None,"9, new cloth marketbhilwara rajasthan. rajasth...",rajasthan,000000,bhilwara,rajasthan,Rajasthan
21,U74999RJ2003PTC018076,VELOCITY TOLLS PRIVATE LIMITED,RoC-Jaipur,018076,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,29/09/2005,31/03/2005,Strike Off,None,"ist floor somani buildingloha mandi, s.c.link ...",rajasthan,000000,mandi,rajasthan,Rajasthan
23,U80903RJ2003PTC018077,SPECTRUM COACHING PRIVATE LIMITED,RoC-Jaipur,018077,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,28/09/2013,31/03/2013,Strike Off,None,"68/283, pratap nagar,sanganer housing board, j...",rajasthan,000000,jaipur,rajasthan,Rajasthan
30,U74300DL2003PTC118785,ARSH PUBLICITY PRIVATE LIMITED,RoC-Delhi,118785,Company limited by Shares,Non-govt company,Private,200000,100000,0,...,30/09/2023,31/03/2023,Active,None,shop no-18gandhi market minto road cp new delh...,delhi,000000,new delhi,delhi,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344913,U24232GJ2003PTC041807,SAILIF MULTICARE PRIVATE LIMITED,RoC-Ahmedabad,041807,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,-,-,Strike Off,None,"sneh setu , nr.jamali hall,vohra ground godhar...",gujarat,000000,panch mahals,gujarat,Gujarat
344928,U27106MH2003PTC138559,WADA ROLLING MILL PRIVATE LIMITED,RoC-Mumbai,138559,Company limited by Shares,Non-govt company,Private,500000,100000,0,...,-,-,Strike Off,None,26 mahaveer chambersnavapur road bhoisar thane...,maharashtra,000000,thane,maharashtra,Maharashtra
344933,U52520TG2003PTC040242,KALUGA TURBO MACHINARY PRIVATE LIMITED,RoC-Hyderabad,040242,Company limited by Shares,Non-govt company,Private,1000000,0,0,...,-,-,Active,None,"11-21-111, ntr nagar,saroor nagar, hyderabad-7...",telangana,000000,hyderabad,telangana,Telangana
344935,U21012HP2003PTC025704,HARIPUR PAPERS PRIVATE LIMITED,RoC-HimachalPradesh,025704,Company limited by Shares,Non-govt company,Private,500000,0,0,...,-,-,Strike Off,None,"haripur road, vill.batehar,bortiwala, distt. s...",himachal pradesh,000000,solan,himachal pradesh,Himachal Pradesh


In [373]:
df_final=df_final[df_final['pincode']!='000000']

In [374]:
df_final

,CIN,state,district,pincode
0,U24239DL2003PTC118783,Delhi,New Delhi,110016
1,U70101MH2003PTC139010,Maharashtra,Mandi,400053
2,U30007MH2003PTC139009,Maharashtra,Mumbai,400049
3,U27109MH2003PTC138995,Maharashtra,Mumbai,400072
4,U28129MH2003PTC138997,Maharashtra,Mumbai,400090
...,...,...,...,...
344946,U18109WB2009PTC134768,West Bengal,Kolkata,700010
344947,U22210OR2009PTC010703,Odisha,None,751015
344948,U29268MH2009PTC193932,Maharashtra,Dhar,421503
344949,U31900DL2009PTC196254,Delhi,Karnal,110033


In [376]:
empty_pin['pincode']=empty_pin['Registered_Address_temp'].apply(lambda x: '|'.join(re.findall(r'\d{3}\s?\d{3}',x))if isinstance(x,str) else None)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9432\3428253604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_pin['pincode']=empty_pin['Registered_Address_temp'].apply(lambda x: '|'.join(re.findall(r'\d{3}\s?\d{3}',x))if isinstance(x,str) else None)


In [377]:
empty_pin['pincode']=empty_pin['pincode'].apply(lambda x: x.split('|')[0] if isinstance(x,str) else x)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9432\3616713687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_pin['pincode']=empty_pin['pincode'].apply(lambda x: x.split('|')[0] if isinstance(x,str) else x)


In [378]:
empty_pin['pincode']=empty_pin['pincode'].apply(lambda x: None if isinstance(x,str) and len(x)==0 else x)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9432\3076698956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_pin['pincode']=empty_pin['pincode'].apply(lambda x: None if isinstance(x,str) and len(x)==0 else x)


In [379]:
empty_pin['pincode']=empty_pin['pincode'].str.replace(' ','')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9432\439403499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_pin['pincode']=empty_pin['pincode'].str.replace(' ','')


In [380]:
set(empty_pin['pincode'])

{'400702',
 '400098',
 '560037',
 '673305',
 '685584',
 '380016',
 '400019',
 '560011',
 '686673',
 '600073',
 '500020',
 '380008',
 '361006',
 '400705',
 '629001',
 '605005',
 '642001',
 '600078',
 '625002',
 '401203',
 '685553',
 '679306',
 '521230',
 '440008',
 '403601',
 '411030',
 '359360',
 '522022',
 '601302',
 '410218',
 '396009',
 '641037',
 '560097',
 '411003',
 '680508',
 '311001',
 '128129',
 '400025',
 '530026',
 '380013',
 '521180',
 '396191',
 '600091',
 '683501',
 '577401',
 '600016',
 '400610',
 '630001',
 '360001',
 '524201',
 '683513',
 '390005',
 '638183',
 '000030',
 '609603',
 '678010',
 '605008',
 '690518',
 '380015',
 '629165',
 '394602',
 '624802',
 '575011',
 '500195',
 '530003',
 '360002',
 '682024',
 '985100',
 '302016',
 '686013',
 '400034',
 '400063',
 '501502',
 '522011',
 '391243',
 '680587',
 '400603',
 '800016',
 '560094',
 '832104',
 '126127',
 '683572',
 '441111',
 '535547',
 '600086',
 '641004',
 '400078',
 '682016',
 '530044',
 '500095',
 '626123',

In [381]:
still_empty=empty_pin[empty_pin['pincode'].str.contains('000000',na=True)]

In [382]:
still_empty

,CIN,Company_Name,ROC_Code,Registration_Number,Company_Category,Company_SubCategory,Class_of_Company,Authorised_Capital_Rs,Paid_up_Capital_Rs,Number_of_Members,...,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status_for_efiling,CrDt,Registered_Address_temp,state,pincode,district,state_backup,state_new
18,U51505MP2003PLC015556,ZENET CONCEPTS INDIA LIMITED,RoC-Gwalior,015556,Company limited by Shares,Non-govt company,Public,500000,244000,0,...,-,-,Strike Off,None,"m-1,mayur plaza,mayur market thatipur gwalior ...",madhya pradesh,000000,gwalior,madhya pradesh,Madhya Pradesh
20,U18101RJ2003PTC018079,SILVER FAB SUITING PRIVATE LIMITED,RoC-Jaipur,018079,Company limited by Shares,Non-govt company,Private,20000000,15305000,0,...,30/09/2024,31/03/2024,Active,None,"9, new cloth marketbhilwara rajasthan. rajasth...",rajasthan,000000,bhilwara,rajasthan,Rajasthan
21,U74999RJ2003PTC018076,VELOCITY TOLLS PRIVATE LIMITED,RoC-Jaipur,018076,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,29/09/2005,31/03/2005,Strike Off,None,"ist floor somani buildingloha mandi, s.c.link ...",rajasthan,000000,mandi,rajasthan,Rajasthan
23,U80903RJ2003PTC018077,SPECTRUM COACHING PRIVATE LIMITED,RoC-Jaipur,018077,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,28/09/2013,31/03/2013,Strike Off,None,"68/283, pratap nagar,sanganer housing board, j...",rajasthan,000000,jaipur,rajasthan,Rajasthan
30,U74300DL2003PTC118785,ARSH PUBLICITY PRIVATE LIMITED,RoC-Delhi,118785,Company limited by Shares,Non-govt company,Private,200000,100000,0,...,30/09/2023,31/03/2023,Active,None,shop no-18gandhi market minto road cp new delh...,delhi,000000,new delhi,delhi,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344900,U51909MH2003PTC138497,SATYAM CONSUMER SERVICES PRIVATE LIMITED,RoC-Mumbai,138497,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,-,-,Strike Off,None,a/p-jamwadital. & dist-jalna jalna mh 000000 in,maharashtra,000000,jalna,maharashtra,Maharashtra
344928,U27106MH2003PTC138559,WADA ROLLING MILL PRIVATE LIMITED,RoC-Mumbai,138559,Company limited by Shares,Non-govt company,Private,500000,100000,0,...,-,-,Strike Off,None,26 mahaveer chambersnavapur road bhoisar thane...,maharashtra,000000,thane,maharashtra,Maharashtra
344933,U52520TG2003PTC040242,KALUGA TURBO MACHINARY PRIVATE LIMITED,RoC-Hyderabad,040242,Company limited by Shares,Non-govt company,Private,1000000,0,0,...,-,-,Active,None,"11-21-111, ntr nagar,saroor nagar, hyderabad-7...",telangana,000000,hyderabad,telangana,Telangana
344935,U21012HP2003PTC025704,HARIPUR PAPERS PRIVATE LIMITED,RoC-HimachalPradesh,025704,Company limited by Shares,Non-govt company,Private,500000,0,0,...,-,-,Strike Off,None,"haripur road, vill.batehar,bortiwala, distt. s...",himachal pradesh,000000,solan,himachal pradesh,Himachal Pradesh


In [383]:
empty_pin=empty_pin[~empty_pin['pincode'].isnull()]

In [385]:
empty_pin=empty_pin[empty_pin['pincode']!='000000']

In [386]:
empty_pin

,CIN,Company_Name,ROC_Code,Registration_Number,Company_Category,Company_SubCategory,Class_of_Company,Authorised_Capital_Rs,Paid_up_Capital_Rs,Number_of_Members,...,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status_for_efiling,CrDt,Registered_Address_temp,state,pincode,district,state_backup,state_new
134,U51909AP2003PTC040439,NARISETTY EXPORTS AND IMPORTS PRIVATE LIMITED,RoC-Vijayawada,040439,Company limited by Shares,Non-govt company,Private,500000,500,0,...,-,-,Strike Off,None,"no.397/a, g.t.road,chowda varam (post), guntur...",andhra pradesh,522019,guntur,andhra pradesh,Andhra Pradesh
135,U64202AP2003PTC040440,RAYALASEEMA ONLINE PRIVATE LIMITED,RoC-Vijayawada,040440,Company limited by Shares,Non-govt company,Private,100000,10000,0,...,-,-,Strike Off,None,"7/243, 2 ngo colony,cuddapah-516 002. cuddapah...",andhra pradesh,516002,None,andhra pradesh,Andhra Pradesh
161,U72900AP2003PTC040436,OM TECHNODIAL PRIVATE LIMITED,RoC-Vijayawada,040436,Company limited by Shares,Non-govt company,Private,100000,10000,0,...,-,-,Strike Off,None,"40/790-3, nehru nagar,kurnool-518 002. kurnool...",andhra pradesh,518002,kurnool,andhra pradesh,Andhra Pradesh
312,U45201KL2003PTC015935,SHADE ENGINEERING PRIVATE LIMITED,RoC-Ernakulam,015935,Company limited by Shares,Non-govt company,Private,400000,0,0,...,-,-,Strike Off,None,"648(5/258),bimal bhavan,mekkad p.o., cheriavap...",kerala,683589,ernakulam,kerala,Kerala
343,U15132GJ2003PTC041963,BABA BEVERAGES PRIVATE LIMITED,RoC-Ahmedabad,041963,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,-,-,Strike Off,None,"12/13, rashmi appartments,opp.akashwani,bhuj p...",gujarat,370001,kachchh,gujarat,Gujarat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344628,U72300KA2003PTC031973,INCHIP NETVISION PRIVATE LIMITED,RoC-Bangalore,031973,Company limited by Shares,Non-govt company,Private,500000,0,0,...,-,-,Under Process of Striking Off,None,"no.23, 1st main,8th cross prashanthnagr bangal...",karnataka,560079,bangalore,karnataka,Karnataka
344631,U72200AP2003PTC041039,VEGI'S COMPUTERS PRIVATE LIMITED,RoC-Vijayawada,041039,Company limited by Shares,Non-govt company,Private,100000,10000,0,...,-,-,Strike Off,None,"2nd floor, y.n.reddy complex,2-66, b.c.m road ...",andhra pradesh,521230,krishna,andhra pradesh,Andhra Pradesh
344667,U19111TN2003PTC050970,ORANGE LEATHER AND FUR PRIVATE LIMITED,RoC-Chennai,050970,Company limited by Shares,Non-govt company,Private,100000,100000,0,...,-,-,Strike Off,None,"3/476, barathidasan street,shanmugam nagar mad...",tamil nadu,600116,anand,tamil nadu,Tamil Nadu
344839,U18101TN2002PTC050114,SRI KAYESAR SPINNERS PRIVATE LIMITED,RoC-Chennai,050114,Company limited by Shares,Non-govt company,Private,500000,100000,0,...,-,-,Strike Off,None,"sri renga bhavanam,no.539, tenkasi road, rajap...",tamil nadu,626117,tirunelveli,tamil nadu,Tamil Nadu


In [387]:
empty_pin=empty_pin[['CIN', 'state_new', 'district', 'pincode']]

In [388]:
empty_pin=empty_pin.rename(columns={'state_new':'state'})

In [389]:
empty_pin

,CIN,state,district,pincode
134,U51909AP2003PTC040439,Andhra Pradesh,guntur,522019
135,U64202AP2003PTC040440,Andhra Pradesh,None,516002
161,U72900AP2003PTC040436,Andhra Pradesh,kurnool,518002
312,U45201KL2003PTC015935,Kerala,ernakulam,683589
343,U15132GJ2003PTC041963,Gujarat,kachchh,370001
...,...,...,...,...
344628,U72300KA2003PTC031973,Karnataka,bangalore,560079
344631,U72200AP2003PTC041039,Andhra Pradesh,krishna,521230
344667,U19111TN2003PTC050970,Tamil Nadu,anand,600116
344839,U18101TN2002PTC050114,Tamil Nadu,tirunelveli,626117


In [390]:
df_final=pd.concat([df_final,empty_pin],axis=0)

In [393]:
df_final

,CIN,state,district,pincode
0,U24239DL2003PTC118783,Delhi,New Delhi,110016
1,U70101MH2003PTC139010,Maharashtra,Mandi,400053
2,U30007MH2003PTC139009,Maharashtra,Mumbai,400049
3,U27109MH2003PTC138995,Maharashtra,Mumbai,400072
4,U28129MH2003PTC138997,Maharashtra,Mumbai,400090
...,...,...,...,...
344628,U72300KA2003PTC031973,Karnataka,Bangalore,560079
344631,U72200AP2003PTC041039,Andhra Pradesh,Krishna,521230
344667,U19111TN2003PTC050970,Tamil Nadu,Anand,600116
344839,U18101TN2002PTC050114,Tamil Nadu,Tirunelveli,626117


In [392]:
df_final['district']=df_final['district'].str.title()

In [ ]:
still_empty['pincode']=still_empty['Registered_Address_temp'].apply(lambda x: )

In [405]:
set(still_empty['Registered_Address_temp'])

{'5-8-327/b/5,public garden rd, hyderabad-1 tg 000000 in',
 'plot no>203 gayatri gardens 12-13-377 lane no>! st>no>@ tarn aka secbad tg 000000 in',
 'vill & p o anangpurfaridabad near suraj kund haryana hr 000000 in',
 '290 hanuman lanedharampeth nagpur mh 000000 in',
 'h.no: 2-48, nethaji colonyfathenagar hyderabad hyderabad tg 000000 in',
 '15/96civil lines kanpur uttar pradesh up 000000 in',
 '972 rajpura roadcivil lines ludhiana punjab pb 000000 in',
 'd-15/174 sector 3rohini new delhi dl 000000 in',
 "fifth floor,'sudharsan building' 86, chamiers road, chennai. chennai tn 000000 in",
 '89a, mettupalayam road,opp housing unit, kavundampalayam, coimbatore-30 coimbatore tn 000000 in',
 '#3235behind gurdwara rajpura distt patiala punjab pb 000000 in',
 '84/60anwar ganj kanpur uttar pradesh up 000000 in',
 '14, iiird floor, dharma towersold no.150 new no.88, nelson manickam road, chennai - 29. chennai tn 000000 in',
 'chikkajantikal,gangavathi (taluk) gangavathi (taluk) ka 000000 in',


In [408]:
import pandas as pd
import re
from sqlalchemy import create_engine
from urllib.parse import quote
engine = create_engine("mysql://altrr_db:%s@192.168.0.32:3306/fdb"% quote("Altrr@2025"))

In [413]:
df=pd.read_sql('select * from cin_address_split',engine)
df

,CIN,state,district,pincode
0,U74993UP2018PTC101160,Uttar Pradesh,Sultanpur,228001
1,U74993UP2018PTC101268,Uttar Pradesh,Meerut,250221
2,U74993UP2018PTC101272,Uttar Pradesh,Mainpuri,205301
3,U74993UP2018PTC101439,Uttar Pradesh,Ballia,277001
4,U74994AP2018PTC107703,Andhra Pradesh,Guntur,522236
...,...,...,...,...
1679524,U72300KA2003PTC031973,Karnataka,Bangalore,560079
1679525,U72200AP2003PTC041039,Andhra Pradesh,Krishna,521230
1679526,U19111TN2003PTC050970,Tamil Nadu,Anand,600116
1679527,U18101TN2002PTC050114,Tamil Nadu,Tirunelveli,626117


In [414]:
df=df.drop_duplicates(subset='CIN')
df

,CIN,state,district,pincode
0,U74993UP2018PTC101160,Uttar Pradesh,Sultanpur,228001
1,U74993UP2018PTC101268,Uttar Pradesh,Meerut,250221
2,U74993UP2018PTC101272,Uttar Pradesh,Mainpuri,205301
3,U74993UP2018PTC101439,Uttar Pradesh,Ballia,277001
4,U74994AP2018PTC107703,Andhra Pradesh,Guntur,522236
...,...,...,...,...
1679515,U72200RJ2003PTC018067,Rajasthan,Udaipur,313004
1679516,U51506TZ2003PTC010466,Tamil Nadu,Coimbatore,641002
1679517,U72200KL2003PTC015930,Kerala,Ernakulam,682018
1679518,U29299RJ2003PTC018080,Rajasthan,Alwar,301001


In [417]:
a=df_final[df_final['district'].isnull()]

,CIN,state,district,pincode
11,U65910DL2003PTC118788,Delhi,None,110092
25,U18101UP2003PTC027220,Uttar Pradesh,None,208002
35,U01111PB2009PTC033311,Punjab,None,140307
42,U25209DL2003PTC118797,Delhi,None,110092
58,U51909GJ2003PTC041949,Gujarat,None,380052
...,...,...,...,...
343694,U24233GJ2003PTC041885,Gujarat,None,382445
343810,U52100GJ2003PTC041898,Gujarat,None,382430
343902,U05004AP2003PTC040372,Andhra Pradesh,None,523155
344121,U51909GJ2003PTC041930,Gujarat,None,380052


In [406]:
fdb = create_engine("mysql://altrr_db:%s@192.168.0.32:3306/fdb"% quote("Altrr@2025"))

In [407]:
df_final.to_sql('cin_address_split',fdb,if_exists='append',index=False)

331912

In [37]:
df2=df1[~df1['CIN'].isin(df['CIN'])]
df2

,CIN,Company_Name,ROC_Name,Registration_Number,Date_of_Incorporation,Email_Id,Registered_Address,Address_at_which_the_books_of_account_are_to_be_maintained,Listed_in_Stock_Exchange,Category_of_Company,...,Paid_up_Capital_Rs,Date_of_last_AGM,Date_of_Balance_Sheet,Company_Status,Jurisdiction,ROC_name_and_office,RD _name_and_Region,Number_of_Members,CrDt,id
12,U74994DL2018PTC329363,HGN TRADERS PRIVATE LIMITED,ROC Delhi,329363,12-02-2018,AADILKRJ12@GMAIL.COM,"F 223 A GROUND FLOOR MANGAL BAZAR, East Delhi,...",-,No,Company limited by shares,...,"5,00,000",20-09-2023,31-03-2023,Active,None,ROC Delhi,"RD, Northern Region",None,2024-09-03 19:35:46,13
16,U74994DL2018PTC329657,FRISCO OVERSEAS PRIVATE LIMITED,ROC Delhi,329657,20-02-2018,mail@frisco.co.in,"House No: 27, 2nd Floor, Najafgarh Road Block-...",-,No,Company limited by shares,...,"15,52,00,000",30-09-2023,31-03-2023,Active,None,ROC Delhi,"RD, Northern Region",None,2024-09-03 19:35:46,17
24,U74994GJ2018PTC100864,RITTOX IT SERVICE PRIVATE LIMITED,ROC Ahmedabad,100864,17-02-2018,ramesh.marand@gmail.com,"5, ELEGANCE BUNGLOWS OPP. BHAVIN SCHOOL, THALT...",-,No,Company limited by shares,...,"1,00,000",-,-,Strike Off,None,ROC Ahmedabad,"RD, North Western Region",None,2024-09-03 19:35:46,25
25,U74994GJ2018PTC100914,INFOCUSP INNOVATIONS PRIVATE LIMITED,ROC Ahmedabad,100914,21-02-2018,nisarg.vyas@gmail.com,"301 to 310 and 401 to 404, 3rd and 4th floor, ...",-,No,Company limited by shares,...,"1,00,000",30-09-2022,31-03-2022,Converted to LLP,None,ROC Ahmedabad,"RD, North Western Region",None,2024-09-03 19:35:46,26
67,U74994PB2018PTC047460,LAKSHAY WORKPOWER PRIVATE LIMITED,ROC Chandigarh,47460,19-02-2018,indu3217@yahoo.com,"HOUSE NO. 92, WARD NO. 9, BHABAT, S.A.S. NAGAR...",-,No,Company limited by shares,...,"1,00,000",30-09-2019,31-03-2019,Active,None,ROC Chandigarh,"RD, Northern Region",None,2024-09-03 19:35:46,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561557,U80301TN2011PTC083618,TECHNICON E-SERVE PRIVATE LIMITED,RoC-Chennai,083618,16/12/2011,techniconeservepltd@gmail.com,"NO. 4/7, NORTH AVANUE ROAD, MUTHAMIL NAGAR, KO...",NaN,Unlisted,Company limited by Shares,...,100000,30/08/2024,31/03/2024,Active,NaN,-,NaN,0,2024-10-05 05:32:59,2398154
1561559,U74999WB2011PTC170775,K. N. FINANCIAL CONSULTANCY COMPANY PRIVATE LI...,RoC-Kolkata,170775,16/12/2011,apu_mondal07@yahoo.co.in,"SONARPUR NOAPARA, 3RD LANE P.O. & P.S. SONARPU...",NaN,Unlisted,Company limited by Shares,...,100000,-,-,Strike Off,NaN,-,NaN,0,2024-10-05 05:33:00,2398158
1561568,U72300MH2011PTC225029,YIPPEE TECHNOLOGY PRIVATE LIMITED,RoC-Mumbai,225029,16/12/2011,kalyandani@gmail.com,"576, Narayan Nagar, Muktainagar, Tal - Muktain...",NaN,Unlisted,Company limited by Shares,...,100000,30/09/2023,31/03/2023,Active,NaN,-,NaN,0,2024-10-05 05:33:07,2398179
1561594,U51909TG2011PTC078075,GIRIDHARI ENTERPRISES PRIVATE LIMITED,RoC-Hyderabad,078075,16/12/2011,giridhari.nokia@gmail.com,"PLOT NO. 48, RAMARAO NAGAR COLONY BESIDE PAPAR...",NaN,Unlisted,Company limited by Shares,...,7500000,30/09/2023,31/03/2023,Active,NaN,-,NaN,0,2024-10-05 05:33:27,2398236
